# revelance ranking

In [1]:
import pandas as pd
from tqdm import tqdm
import pickle
tqdm.pandas()

In [2]:
import json

with open('C:/Users/ngoph/Desktop/zalo_ai/wikipedia_20220620_cleaned/wikipedia_20220620_cleaned.jsonl', 'r',encoding='utf-8') as json_file:
    json_list = list(json_file)

In [3]:
json_list = [json.loads(json_str) for json_str in json_list]

In [4]:
df = pd.DataFrame(json_list)
df.head()

,id,url,title,text,timestamp,revid
0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s...",2022-05-12T12:46:53Z,68591979
1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society\n\nInternet Society hay ISOC ...,2022-01-20T07:59:10Z,67988747
2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt\n\nTiếng Việt, cũng gọi là tiếng Vi...",2022-05-29T03:42:42Z,68660631
3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio\n\nOhio (viết tắt là OH, viết tắt cũ là O...",2022-04-17T08:15:22Z,68482118
4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California\n\nCalifornia (phát âm như ""Ca-li-p...",2022-06-16T15:27:07Z,68738039


In [5]:
# free up memory
del json_list

In [6]:
from bs4 import BeautifulSoup
import re

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def normalize_text(text):
    text = strip_html_tags(text)
    # remove \n 
    text = re.sub(r'\n', ' ', text)
    # remove \xa0
    text = re.sub(r'\xa0', ' ', text)
    # remove 'BULLET::::-'
    text = re.sub(r'BULLET::::-', ' ', text)
    # remove = if more than 1
    text = re.sub(r'={2,}', ' ', text)
    # remove duplicate spaces
    text = re.sub(r'\s+', ' ', text)

    return text

In [7]:
df['text'] = df['text'].progress_apply(normalize_text)

  0%|          | 291/1273469 [00:00<28:55, 733.43it/s]c:\Users\ngoph\.conda\envs\research\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
  0%|          | 4646/1273469 [00:02<09:40, 2186.11it/s]c:\Users\ngoph\.conda\envs\research\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
100%|██████████| 1273469/1273469 [01:33<00:00, 13675.85it/s]


In [8]:
df.to_csv('data/zac2022_train_merged_final_preprocessing.json')

In [9]:
df.head()

,id,url,title,text,timestamp,revid
0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,Trang Chính __NOEDITSECTION__,2022-05-12T12:46:53Z,68591979
1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society Internet Society hay ISOC là ...,2022-01-20T07:59:10Z,67988747
2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt Tiếng Việt, cũng gọi là tiếng Việt ...",2022-05-29T03:42:42Z,68660631
3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio Ohio (viết tắt là OH, viết tắt cũ là O.) ...",2022-04-17T08:15:22Z,68482118
4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California California (phát âm như ""Ca-li-phót...",2022-06-16T15:27:07Z,68738039


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
corpus = df['text'].tolist()

In [12]:
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(corpus)
# pickle.dump(X, open('X.pkl', 'wb'))
# filename = 'tfidfVectorizer.pkl'
# pickle.dump(vectorizer, open(filename, 'wb'))

In [13]:
# use this code to load
vectorizer = pickle.load(open('tfidfVectorizer.pkl', 'rb'))
X = pickle.load(open('X.pkl', 'rb'))
# X = vectorizer.transform(corpus)

In [14]:
vectorizer.get_feature_names_out()

array(['00', '000', '0000', ..., '𪤄葛鐄', '𪼀越南', '𬇕尾'], dtype=object)

# test 

In [15]:
question = 'người thứ 45 làm tổng thống ở hoa kỳ  là ai'
q = vectorizer.transform([question])

# get top 10 similar documents
top_k = 10
top_k_idx = X.dot(q.T).toarray().flatten().argsort()[-top_k:][::-1]
top_k_idx

array([  37692,  130135, 1256127,  145255,   96757,    1094, 1166044,
       1252269, 1133463,   22507], dtype=int64)

In [16]:
df.iloc[top_k_idx]

,id,url,title,text,timestamp,revid
37692,213603,https://vi.wikipedia.org/wiki?curid=213603,Danh sách Tổng thống Hoa Kỳ,Danh sách Tổng thống Hoa Kỳ Theo Hiến pháp Hoa...,2022-06-02T13:24:10Z,68676832
130135,540903,https://vi.wikipedia.org/wiki?curid=540903,45,45,2021-12-15T12:04:30Z,67577286
1256127,19556409,https://vi.wikipedia.org/wiki?curid=19556409,Tổng thống Ai Cập,Tổng thống Ai Cập Tổng thống Ai Cập (tiếng Ả R...,2021-12-20T13:19:58Z,67698017
145255,574342,https://vi.wikipedia.org/wiki?curid=574342,Thứ tự kế vị Tổng thống Hoa Kỳ,Thứ tự kế vị Tổng thống Hoa Kỳ Thứ tự kế vị Tổ...,2022-04-11T01:06:12Z,68442205
96757,406328,https://vi.wikipedia.org/wiki?curid=406328,Phó Tổng thống Hoa Kỳ,"Phó Tổng thống Hoa Kỳ Phó Tổng thống Hoa Kỳ (""...",2022-04-11T01:20:06Z,68442260
1094,5645,https://vi.wikipedia.org/wiki?curid=5645,Tổng thống Hoa Kỳ,Tổng thống Hoa Kỳ Tổng thống Hoa Kỳ là nguyên ...,2022-05-17T13:32:16Z,68611311
1166044,13469614,https://vi.wikipedia.org/wiki?curid=13469614,Danh sách Phó Tổng thống Hoa Kỳ,Danh sách Phó Tổng thống Hoa Kỳ Đã có 49 Phó T...,2022-06-02T13:24:35Z,68676833
1252269,19514530,https://vi.wikipedia.org/wiki?curid=19514530,Tổng thống đắc cử của Hoa Kỳ,Tổng thống đắc cử của Hoa Kỳ Tổng thống đắc cử...,2021-08-07T10:59:33Z,65358906
1133463,3276845,https://vi.wikipedia.org/wiki?curid=3276845,Phó Tổng thống Nam Phi,Phó Tổng thống Nam Phi Phó Tổng thống Nam Phi ...,2021-11-11T16:15:36Z,66847734
22507,119100,https://vi.wikipedia.org/wiki?curid=119100,Tổng thống Nga,Tổng thống Nga Tổng thống Nga () là nguyên thủ...,2022-06-11T14:04:59Z,68708406


In [17]:
from transformers import pipeline
qa = pipeline('question-answering', model='model_2\checkpoint-1000',
    device=0
)

In [18]:
import tensorflow as tf
model_1 = tf.keras.models.load_model('model_1')
LABEL={
	'wiki':0,
	'number':1,
	'text':2
}

In [19]:
context = df.iloc[1146916]['text']
qa(question, context)

{'score': 0.9908002018928528, 'start': 24, 'end': 36, 'answer': 'Donald Trump'}

In [31]:
# df[df['text'].str.lower().str.contains('tổng thống thứ 45')]

In [21]:
inputs = []
for i in top_k_idx:
    row = df.iloc[i]
    context = row['text']
    inputs += [{'question': question, 'context': context}]

outputs = qa(inputs)
print(question)
type=model_1.predict([question],verbose=0).argmax()
print(list(LABEL.keys())[type])
for i,output in enumerate(outputs):
    print(df.iloc[top_k_idx[i]].title)
    print(output)
    print('----')

người thứ 45 làm tổng thống ở hoa kỳ  là ai
wiki
Danh sách Tổng thống Hoa Kỳ
{'score': 0.9690871238708496, 'start': 2183, 'end': 2197, 'answer': 'Andrew Jackson'}
----
45
{'score': 2.3557308850286063e-06, 'start': 0, 'end': 2, 'answer': '45'}
----
Tổng thống Ai Cập
{'score': 4.947704383084783e-06, 'start': 964, 'end': 977, 'answer': 'Omar Suleiman'}
----
Thứ tự kế vị Tổng thống Hoa Kỳ
{'score': 0.9653265476226807, 'start': 6656, 'end': 6666, 'answer': 'John Tyler'}
----
Phó Tổng thống Hoa Kỳ
{'score': 0.9853323101997375, 'start': 18379, 'end': 18389, 'answer': 'John Adams'}
----
Tổng thống Hoa Kỳ
{'score': 0.9969035387039185, 'start': 1142, 'end': 1151, 'answer': 'Joe Biden'}
----
Danh sách Phó Tổng thống Hoa Kỳ
{'score': 2.0650209989980794e-05, 'start': 41, 'end': 62, 'answer': 'Phó Tổng thống Hoa Kỳ'}
----
Tổng thống đắc cử của Hoa Kỳ
{'score': 1.0518874660192523e-07, 'start': 453, 'end': 462, 'answer': 'Joe Biden'}
----
Phó Tổng thống Nam Phi
{'score': 0.00013176647189538926, 'start

# inference

In [22]:
with open('data/zac2022_testa_only_question.json',encoding='utf-8') as f:
    data=json.load(f)
    reference_df = pd.json_normalize(data,'data')

In [23]:
text_file = open("vietnamese-stopwords.txt", "r")
stopwords = text_file.read().split('\n')

In [24]:
def context_remove_stopwords(context):
    context = context.split(' ')
    context = [word for word in context if word not in stopwords]
    context = ' '.join(context)
    return context

In [43]:
def date_format(answer):
    if re.search(r'ngày \d{1,2} tháng \d{1,2} năm \d{4}',answer):
        answer = re.search(r'ngày \d{1,2} tháng \d{1,2} năm \d{4}',answer).group()
    # ngay xx thang xx
    elif re.search(r'ngày \d{1,2} tháng \d{1,2}',answer):
        answer = re.search(r'ngày \d{1,2} tháng \d{1,2}',answer).group()
    # thang xx nam xxx
    elif re.search(r'tháng \d{1,2} năm \d{4}',answer):
        answer = re.search(r'tháng \d{1,2} năm \d{4}',answer).group()
    # nam xxxx
    elif re.search(r'năm \d{4}',answer):
        answer = re.search(r'năm \d{4}',answer).group()

    # 1/1/2021  dd{token}mm{token}yyyy
    elif re.search(r'\d{1,2}[ /|-]\d{1,2}[ /|-]\d{4}',answer):
        answer = re.search(r'\d{1,2}[ /|-]\d{1,2}[ /|-]\d{4}',answer).group()
        token = re.search(r'[ /|-]',answer).group()
        answer = answer.split(token)
        answer = 'ngày '+answer[0]+' tháng '+answer[1]+' năm '+answer[2]
    # 1/1       dd{token}mm
    elif re.search(r'\d{1,2}[ /|-]\d{1,2}',answer):
        answer = re.search(r'\d{1,2}[ /|-]\d{1,2}',answer).group()
        token = re.search(r'[ /|-]',answer).group()
        answer = answer.split(token)
        answer = 'ngày '+answer[0]+' tháng '+answer[1]
    # 1/2021    mm{token}yyyy
    elif re.search(r'\d{1,2}[ /|-]\d{4}',answer):
        answer = re.search(r'\d{1,2}[ /|-]\d{4}',answer).group()
        token = re.search(r'[ /|-]',answer).group()
        answer = answer.split(token)
        answer = 'tháng '+answer[0]+' năm '+answer[1]
    return answer

In [44]:
answer = 'asd á ngày 13 tháng 9 năm 1943 abc d'
answer = '3 10 2021'
date_format(answer)

'ngày 3 tháng 10 năm 2021'

In [45]:
submission = []

for row in tqdm(reference_df.iterrows(),total=len(reference_df)):
    question = row[1]['question']
    q = vectorizer.transform([question])
    top_k = 10
    top_k_idx = X.dot(q.T).toarray().flatten().argsort()[-top_k:][::-1]
    result = []
    for i in top_k_idx:
        doc_row = df.iloc[i]
        context = doc_row['text']
        context = context_remove_stopwords(context)
        result.append((doc_row['title'], qa(question=question, context=context)))

    result.sort(key=lambda x: x[1]['score'], reverse=True)
    best_result = result[0]
    type=model_1.predict([question],verbose=0).argmax()
    if type==LABEL['wiki']:
        answer='wiki/'+best_result[1]['answer'].replace(' ','_')
    if type == LABEL['number']:
        answer=best_result[1]['answer']
        answer = re.sub(r'\D', '', answer)
    if type == LABEL['text']:
        answer=best_result[1]['answer'].lower()
        answer = date_format(answer)
    if best_result[1]['score']<1e-8:
        answer=None

    submission.append({
        'id': row[1]['id'],
        'question': question,
        'answer': answer,
        'raw_answer': best_result[1]['answer'],
        'type': type
    })

  0%|          | 0/600 [00:00<?, ?it/s]c:\Users\ngoph\.conda\envs\research\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 600/600 [13:36<00:00,  1.36s/it]


In [49]:
df_submission = pd.DataFrame(submission)

In [50]:
df_submission

,id,question,answer,raw_answer,type
0,testa_1,Đạo diễn phim Titanic là ai,wiki/James_Cameron,James Cameron,0
1,testa_2,Tổng thống Hoa Kỳ thứ 45 là ai,wiki/Kamala_Harris,Kamala Harris,0
2,testa_3,Hiện nay ai là tổng bí thư nước Việt Nam,wiki/Đặng_Tiểu_Bình,Đặng Tiểu Bình,0
3,testa_4,đâu là bản hiến pháp lâu đời nhất thế giới,wiki/Hiến_pháp_1946,Hiến pháp 1946,0
4,testa_5,Tổ chức thống nhất châu Phi được thành lập ở đâu,wiki/Ai_Cập,Ai Cập,0
...,...,...,...,...,...
595,testa_596,Tổng thống thứ 100 của nước Mỹ là ai,wiki/Kamala_Harris,Kamala Harris,0
596,testa_597,Tỉnh nào ở Việt Nam giáp với nước Nga,wiki/tỉnh_Thanh_Hóa,tỉnh Thanh Hóa,0
597,testa_598,người việt nam đầu tiên đạt Quả bóng vàng châu...,wiki/Lionel_Messi,Lionel Messi,0
598,testa_599,nguyên tố hoá học nào nhẹ hơn hidro,wiki/hydro_heli,hydro heli,0


In [52]:
json_data = df_submission.to_dict(orient='records')
with open('submission.json', 'w', encoding='utf-8') as f:
    json.dump({'data': json_data}, f, ensure_ascii=False, indent=2)

In [55]:
# drop raw_answer and type
df_submission.drop(['raw_answer','type'],axis=1,inplace=True)
json_data = df_submission.to_dict(orient='records')
with open('submission_final.json', 'w', encoding='utf-8') as f:
    json.dump({'data': json_data}, f, ensure_ascii=False, indent=2)